In [ ]:
import csv
import time
from datetime import datetime, timedelta
import requests

# Base URL for the Open-Meteo API
base_url = "https://archive-api.open-meteo.com/v1/archive"

# Adjust years here
start_year = 2007
end_year = 2024
district_name = 'CP_Kandy' #CP - provincial code (Central Province)


# Coordinates for Sri Lanka (You can adjust this to specific districts if needed)
sri_lanka = {
    "name": "Kandy",
    "latitude": 7.2906,  # Approximate center
    "longitude": 80.6337
}

# Define the CSV file fields
fieldnames = [
    'Week Start Date', 'Week End Date',
    'Avg Max Temp (°C)', 'Avg Min Temp (°C)',
    'Avg Apparent Max Temp (°C)', 'Avg Apparent Min Temp (°C)',
    'Total Precipitation (mm)', 'Total Rain (mm)',
    'Avg Wind Speed (km/h)', 'Max Wind Gusts (km/h)',
    'Weather Code', 'Avg Daylight Duration (hours)',
    'Avg Sunrise Time', 'Avg Sunset Time'
]

# Function to collect data
def collect_data(location, start_date, end_date):
    params = {
        "latitude": location['latitude'],
        "longitude": location['longitude'],
        "start_date": start_date,
        "end_date": end_date,
        "daily": "temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,precipitation_sum,rain_sum,weathercode,sunrise,sunset,wind_speed_10m_max,wind_gusts_10m_max",
        "timezone": "Asia/Colombo"
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None

# Helper function to calculate average time (HH:MM format)
def calculate_average_time(time_list):
    total_seconds = 0
    for time_str in time_list:
        time_only = time_str.split('T')[1]  # Extract the time part after 'T'
        time_obj = datetime.strptime(time_only, "%H:%M")
        total_seconds += time_obj.hour * 3600 + time_obj.minute * 60
    avg_seconds = total_seconds / len(time_list)
    avg_time = datetime.utcfromtimestamp(avg_seconds).strftime("%H:%M")
    return avg_time

# Helper function to calculate daylight duration in hours
def calculate_daylight_duration(sunrise_times, sunset_times):
    durations = []
    for sunrise, sunset in zip(sunrise_times, sunset_times):
        sunrise_time = sunrise.split('T')[1]
        sunset_time = sunset.split('T')[1]
        sunrise_dt = datetime.strptime(sunrise_time, "%H:%M")
        sunset_dt = datetime.strptime(sunset_time, "%H:%M")
        duration = (sunset_dt - sunrise_dt).total_seconds() / 3600.0
        durations.append(duration)
    return sum(durations) / len(durations)

# Loop through the weeks and save data to CSV
with open(f'{district_name}_weekly_data_{start_year}_{end_year}.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    current_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 8, 25)

    while current_date <= end_date:
        next_week = current_date + timedelta(days=7)
        start_date_str = current_date.strftime('%Y-%m-%d')
        end_date_str = next_week.strftime('%Y-%m-%d')
        # end-date correction
        # week_end_date = current_date + timedelta(days=6)
        # end_date_str = week_end_date.strftime('%Y-%m-%d')
        print(f"Collecting data for {sri_lanka['name']} from {start_date_str} to {end_date_str}...")

        data = collect_data(sri_lanka, start_date_str, end_date_str)
        if data:
            # Calculate weekly averages and totals
            avg_max_temp = sum(data['daily']['temperature_2m_max']) / len(data['daily']['temperature_2m_max'])
            avg_min_temp = sum(data['daily']['temperature_2m_min']) / len(data['daily']['temperature_2m_min'])
            avg_apparent_max_temp = sum(data['daily']['apparent_temperature_max']) / len(data['daily']['apparent_temperature_max'])
            avg_apparent_min_temp = sum(data['daily']['apparent_temperature_min']) / len(data['daily']['apparent_temperature_min'])
            total_precipitation = sum(data['daily']['precipitation_sum'])
            total_rain = sum(data['daily']['rain_sum'])
            avg_wind_speed = sum(data['daily']['wind_speed_10m_max']) / len(data['daily']['wind_speed_10m_max'])
            max_wind_gusts = max(data['daily']['wind_gusts_10m_max'])
            weather_code = max(set(data['daily']['weathercode']), key=data['daily']['weathercode'].count)
            avg_sunrise_time = calculate_average_time(data['daily']['sunrise'])
            avg_sunset_time = calculate_average_time(data['daily']['sunset'])
            avg_daylight_duration = calculate_daylight_duration(data['daily']['sunrise'], data['daily']['sunset'])

            row = {
                'Week Start Date': start_date_str,
                'Week End Date': end_date_str,
                'Avg Max Temp (°C)': avg_max_temp,
                'Avg Min Temp (°C)': avg_min_temp,
                'Avg Apparent Max Temp (°C)': avg_apparent_max_temp,
                'Avg Apparent Min Temp (°C)': avg_apparent_min_temp,
                'Total Precipitation (mm)': total_precipitation,
                'Total Rain (mm)': total_rain,
                'Avg Wind Speed (km/h)': avg_wind_speed,
                'Max Wind Gusts (km/h)': max_wind_gusts,
                'Weather Code': weather_code,
                'Avg Daylight Duration (hours)': avg_daylight_duration,
                'Avg Sunrise Time': avg_sunrise_time,
                'Avg Sunset Time': avg_sunset_time
            }
            writer.writerow(row)

        current_date = next_week
        time.sleep(1)  # Adjust based on API rate limits

print(f"Data collection complete. Saved to '{district_name}_data_{start_year}_{end_year}.csv'.")

TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'